In [1]:
!pip3 install requests 


In [2]:
import base64

In [14]:
import requests
import datetime
from urllib.parse import urlencode 
#urlendcode can be wrpaped a round a data dictionayry 
# and will turn it inot something that makes sense in a url


In [4]:
## for envrionment variables 
import dotenv
import sys 
import os
dotenv.load_dotenv()
print(os.getenv("CLIENT_ID"))
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = None
    access_token_did_expire = True 
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base 64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None: 
            raise Exeption("You must set client id and client secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
                "Authorization": f"Basic {client_creds_b64}" #Basic <base64 encoded client_id:client_secret>
        }
    
    def get_token_data(self):
        return{
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        access_time = datetime.datetime.now()
        valid_request = r.status_code in range(200, 299)
        if not valid_request:
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data["access_token"]
        self.access_token = access_token
        expires_in =  data["expires_in"]
        expires = access_time+datetime.timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.perform_auth()

True

In [8]:
access_token = spotify.access_token

In [17]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({
    "q": "Time",
    "type": "track"
})
lookup_url = f"{endpoint}?{data}"
#r = requests.get(endpoint, data=data, headers=headers)
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

200


In [1]:
#r.json()


In [2]:
"""data = urlencode({
    "q": "A Lannister Always pays his debts",
    "type": "track"
})
lookup_url = f"{endpoint}?{data}"
#r = requests.get(endpoint, data=data, headers=headers)
r = requests.get(lookup_url, headers=headers)
r.json()
"""

'data = urlencode({\n    "q": "A Lannister Always pays his debts",\n    "type": "track"\n})\nlookup_url = f"{endpoint}?{data}"\n#r = requests.get(endpoint, data=data, headers=headers)\nr = requests.get(lookup_url, headers=headers)\nr.json()\n'